# Ex1 - Filtering and Sorting Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [11]:
import pandas as pd
from pyspark.sql import SparkSession


spark = SparkSession.builder\
                    .appName('Chipo')\
                    .getOrCreate()

25/05/30 17:16:27 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.92 instead (on interface wlo1)
25/05/30 17:16:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/30 17:16:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/30 17:16:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

### Step 3. Assign it to a variable called chipo.

In [14]:
import requests

url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'

url_request = requests.get(url)

with open('data.csv', 'w', encoding='UTF-8') as f:
   f.write(url_request.text)


df = pd.read_csv('data.csv', sep='\t')

chipo = spark.read.csv('data.csv',  sep='\t', header=True)

### Step 4. How many products cost more than $10.00?

In [16]:
chipo.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, regexp_replace




+--------------------+
|item_price_sin_signo|
+--------------------+
|               2.39 |
|               3.39 |
|               3.39 |
|               2.39 |
|              16.98 |
|              10.98 |
|               1.69 |
|              11.75 |
|               9.25 |
|               9.25 |
|               4.45 |
|               8.75 |
|               8.75 |
|              11.25 |
|               4.45 |
|               2.39 |
|               8.49 |
|               8.49 |
|               2.18 |
|               8.75 |
+--------------------+
only showing top 20 rows



### Step 5. What is the price of each item? 
###### print a data frame with only two columns item_name and item_price

### Step 6. Sort by the name of the item

### Step 7. What was the quantity of the most expensive item ordered?

### Step 8. How many times was a Veggie Salad Bowl ordered?

### Step 9. How many times did someone order more than one Canned Soda?